In [1]:
import yaml
import json
import os
import pandas as pd
import re
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

secrets = json.load(open('../secrets.json'))

learnsets_path = '../data/PokeroleBotPokeLearnMovesFull.csv'
moves_path = '../data/PokeroleBotpokeMoveSorted.csv'
abilities_path = '../data/PokeroleBotPokeRoleAbilities.csv'
items_path = '../data/PokeroleBotPokeRoleItems.csv'
pokedex_path = '../data/PokeroleBotPokeroleStats.csv'
natures_path = '../data/PokeroleBotNatures.csv'

pokedex_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Pokedex/'
abilities_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Abilities/'
moves_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Moves/'
learnsets_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Learnsets/'
natures_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Natures/'

os.makedirs(pokedex_output,exist_ok=True)
os.makedirs(abilities_output,exist_ok=True)
os.makedirs(moves_output,exist_ok=True)
os.makedirs(learnsets_output,exist_ok=True)
os.makedirs(natures_output,exist_ok=True)

/Users/bill/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [275]:
ability_template = '''## `= this.name`

> *`= this.Description`*

**Effect:** `= this.Effect`'''

#-------------------------------------------------------------

moves_template = '''### `= this.name` 
*`= this.Description`*

**Accuracy:** `= this.Accuracy1` + `= this.Accuracy2`
**Damage:** `= this.Power` `= choice(length(this.Damage1)=0, "","\+ "+ this.Damage1)` `= choice(length(this.Damage2)=0, "","\+ "+ this.Damage2)`

| Type          | Target          | Damage Type          | Power          |
| ------------- | --------------- | ---------------- | -------------- |
| `= this.Type` | `= this.Target` | `= this.DmgType` | `= this.Power` | 

**Effect:** `= this.Effect`'''

#-------------------------------------------------------------

learnsets_template = '''## `= this.Name` Learnset

**DexID:** `= this.DexID`

```dataview
TABLE WITHOUT ID
    T[0] AS Learned,
    T[1] AS Move
FROM #PokeroleSRD/Learnsets
flatten moves as T
where file.path = this.file.path
```
'''

#-------------------------------------------------------------------

natures_template = '''## `= this.Nature`

**Confidence**: `= this.Confidence`

*`= this.Keywords`*

> `= this.Description`
'''

# Pokedex Entries

In [252]:
data = pd.read_csv(pokedex_path)
data.columns = data.columns.str.title().str.replace(' ','')
data = data.rename({'No.':'Number', 'Hp': 'BaseHP','Unevolved?':'Unevolved', 'Dexid':'DexID'},axis=1)
data = data.fillna('')
data['BoxSprite'] = 'SRD-BoxSprite-'+data['Sprite']
data['HomeSprite'] = 'SRD-HomeSprite-'+data['Sprite']
for i, row in data.iterrows():
    yam = row.to_dict()
    if not yam['Unevolved']: yam['Unevolved'] = 'No'
    if not yam['Legendary']: yam['Legendary'] = 'No'
    if not yam['GoodStarter']: yam['GoodStarter'] = 'No'
    yam['Learnset'] = f"[[SRD-{yam['Name']}-Learnset]]"
    abilities = (f"[[SRD-{yam['Ability1']}|{yam['Ability1']}]]"
                        f"{'' if not yam['Ability2'] else ' / [[SRD-'+ yam['Ability2']+'|'+yam['Ability2']+']]'}"
                        f"{'' if not yam['HiddenAbility'] else ' ([[SRD-'+yam['HiddenAbility']+'|'+yam['HiddenAbility']+']])'}"
                        f"{'' if not yam['EventAbilities'] else ' <[[SRD-'+yam['EventAbilities']+'|'+yam['EventAbilities']+']]>'}"
                        )
    types = f"{yam['Type1']} / {yam['Type2']}" if yam['Type2'] else yam['Type1']
    INTEGERS = ['BaseHP', 'Strength', 'MaxStrength',
       'Dexterity', 'MaxDexterity', 'Vitality', 'MaxVitality', 'Special',
       'MaxSpecial', 'Insight', 'MaxInsight']
    for key in INTEGERS:
        yam[key] = int(yam[key])

    template = f"""# `= this.name`

![[{yam['HomeSprite']}|right]]

**DexID**:: {yam['DexID']}
**Species**:: {yam['Name']}
**Type**:: {types}
**Abilities**:: {abilities}
**Base HP**:: {yam['BaseHP']}

![[{yam['BoxSprite']}|right]]

|           |                                                                                        |                                          |
| --------- | -------------------------------------------------------------------------------------- | ---------------------------------------- |
| Strength  | `= padleft(padright("",this.MaxStrength - this.Strength,"⭘"),this.MaxStrength,"⬤")`    | (Strength::{yam['Strength']})/(MaxStrength::{yam['MaxStrength']})   |
| Dexterity | `= padleft(padright("",this.MaxDexterity - this.Dexterity,"⭘"),this.MaxDexterity,"⬤")` | (Dexterity:: {yam['Dexterity']})/(MaxDexterity::{yam['MaxDexterity']}) |
| Vitality  | `= padleft(padright("",this.MaxVitality - this.Vitality,"⭘"),this.MaxVitality,"⬤")`    | (Vitality::{yam['Vitality']})/(MaxVitality::{yam['MaxVitality']})   |
| Special   | `= padleft(padright("",this.MaxSpecial - this.Special,"⭘"),this.MaxSpecial,"⬤")`       | (Special::{yam['Special']})/(MaxSpecial::{yam['MaxSpecial']})     |
| Insight   | `= padleft(padright("",this.MaxInsight - this.Insight,"⭘"),this.MaxInsight,"⬤")`       | (Insight::{yam['Insight']})/(MaxInsight::{yam['MaxInsight']})     |

**Recommended Rank**:: {yam['RecommendedRank']}
**Good Starter**:: {yam['GoodStarter']}
**Can Evolve**:: {yam['Unevolved']}

![[SRD-{yam['Name']}-Learnset]]"""
        
    for x in ['DexID','Strength','MaxStrength','Dexterity','MaxDexterity',
              'Vitality','MaxVitality','Special','MaxSpecial','Insight','MaxInsight',
             'BaseHP', 'RecommendedRank', 'GoodStarter', 'Unevolved']:
        del yam[x]
    yaml_output = f"---\n{yaml.dump(yam)}---\n\n#PokeroleSRD/Pokedex\n\n{template}"
#     print(yaml_output)
    open(pokedex_output+f"SRD-{yam['Name']}.md",'w').write(yaml_output)

# Ability Entries

In [93]:
data = pd.read_csv(abilities_path)
data = data.fillna('')
for i, row in data.iterrows():
    yam = row.to_dict()
    yaml_output = f"---\n{yaml.dump(yam)}---\n\n#PokeroleSRD/Abilities\n\n{ability_template}"
#     print(yaml_output)
    open(abilities_output+f"SRD-{yam['Name']}.md",'w').write(yaml_output)

# Move Dex



In [213]:
data = pd.read_csv(moves_path)
data = data.fillna('')
data.Type = data.Type.str.capitalize()
for i, row in data.iterrows():
    yam = row.to_dict()
#     INTEGERS = ['Power']
#     for key in INTEGERS:
#         yam[key] = int(yam[key])
    yaml_output = f"---\n{yaml.dump(yam)}---\n\n#PokeroleSRD/Moves\n\n{moves_template}"
#     print(yaml_output)
#     break
    open(moves_output+f"SRD-{yam['Name']}.md",'w').write(yaml_output)

# Move Lists

In [281]:
data = pd.read_csv(learnsets_path)
data = data.fillna('')
for i, row in data.iterrows():
    yam = {}
    yam['Name'] = row['Name']
    yam['Species'] = f"[[SRD-{row['Name']}|{row['Name']}]]"
    yam['DexID'] = row['No.']
    moves = []
    for i in range(1,29):
        if row[f'Rank{i}']:
            if moves and row[f'Rank{i}'] != moves[-1][0]: 
                moves.append(["---------------------------","---------------------------"])
            m = row[f'Move{i}']
            moves.append([row[f'Rank{i}'],f'[[SRD-{m}|{m}]]'])
    yam['Moves'] = moves
    yaml_output = f"---\n{yaml.dump(yam)}---\n\n#PokeroleSRD/Learnsets\n\n{learnsets_template}"
#     print(yaml_output)
    open(learnsets_output+f"SRD-{yam['Name']}-Learnset.md",'w').write(yaml_output)
    

# Natures

In [278]:
data = pd.read_csv(natures_path,sep=',', skipinitialspace=True)
data = data.fillna('')
for i, row in data.iterrows():
    yam = row.to_dict()
    yaml_output = f"---\n{yaml.dump(yam)}---\n\n#PokeroleSRD/Natures\n\n{natures_template}"
#     print(yaml_output)
    open(natures_output+f"SRD-{yam['Name']}.md",'w').write(yaml_output)

# Scratch

In [7]:
sprites = os.listdir('/Users/bill/Code/special-pancake/In/pokesprite/pokemon-gen8/regular')
data = pd.read_csv(pokedex_path)
BoxSprites = []
for n in data.Name:
    best_match = process.extractOne(n, sprites, scorer=fuzz.token_sort_ratio)
    # print(n, best_match)
    BoxSprites.append(best_match[0])
data['BoxSprite'] = BoxSprites

In [195]:
# x_x_DexNum_Form_Femaled/Maled/MaleFemale_Gigantamax_x_front/back_Normal/Rare
# 0_1_2______3____4________________________5__________6_7__________8
home = sorted(os.listdir('/Users/bill/Code/special-pancake/In/Pokémon HOME/'))
data = pd.read_csv(pokedex_path)

pattern = '^[a-z]+_[a-z]+_(\d+)_(\d+)_([a-z]+)_([a-z]+)_\d+_f_([a-z]+)'
# Group 1 = Number
# Group 2 = Form  
# Group 3 = Gender (mf, md, fd, uk)
# Group 4 = Gigantamax? (n/g)
# Group 5 = Rarity (n/r)

shiny = []
gigantamax = []
regular = []

for f in home:
    m = re.match(pattern,f)
    if not m:
        continue
    if m.group(3) == 'fd':
        continue
    if m.group(4) == 'g':
        gigantamax.append(f)
        continue
    if m.group(5) == 'r':
        shiny.append(f)
        continue
    regular.append(f)
print("Shiny:", len(shiny), "Giga:", len(gigantamax), "Regular:", len(regular))

sprites = pd.Series(regular).to_frame()
sprites['num'] = sprites[0].str.extract('^[a-z]+_[a-z]+_(\d+)_\d+')
sprites['form'] = sprites[0].str.extract('^[a-z]+_[a-z]+_\d+_(\d+)')
sprites['key'] = sprites['num'].astype('str')+sprites['form'].astype('int').astype('str')


data['num'] = data['DexID'].str.extract('^(\d+)')
data['form'] = data['DexID'].str.extract('^\d+[A-Z]+(\d+)')
data.loc[data.DexID.str.isdigit(), 'form'] = '0'
data.loc[data.DexID.str.match('\d+[A-Z]+$'), 'form'] = '1'

# sprites.to_csv('sprites.csv',index=0)
# data.to_csv('output.csv',index=0)

data['key'] = data['num'].astype('str')+data['form'].astype('int').astype('str')

data = data.merge(sprites, on='key')
data.apply(lambda x: shutil.copy('/Users/bill/Code/special-pancake/In/Pokémon HOME/'+x[0], '/Users/bill/Code/special-pancake/In/Pokémon HOME new/SRD-HomeSprite-'+x.Sprite),axis=1)
data.to_csv('output.csv',index=0)

Shiny: 1196 Giga: 66 Regular: 1270


,No.,DexID,Name,Type 1,Type 2,HP,Strength,Max Strength,Dexterity,Max Dexterity,...,Gender Type,Legendary,Sprite,Good Starter,num_x,form_x,key,0,num_y,form_y
0,0,0000,Egg,Normal,NaN,1,1,1,1,1,...,NaN,NaN,egg.png,NaN,0000,0,00000,poke_icon_0000_000_uk_n_00000000_f_n.png,0000,000
1,1,0001,Bulbasaur,Grass,Poison,3,2,4,2,4,...,NaN,NaN,bulbasaur.png,Yes,0001,0,00010,poke_icon_0001_000_mf_n_00000000_f_n.png,0001,000
2,2,0002,Ivysaur,Grass,Poison,4,2,4,2,4,...,NaN,NaN,ivysaur.png,NaN,0002,0,00020,poke_icon_0002_000_mf_n_00000000_f_n.png,0002,000
3,3,0003,Venusaur,Grass,Poison,5,2,5,2,5,...,NaN,NaN,venusaur.png,NaN,0003,0,00030,poke_icon_0003_000_md_n_00000000_f_n.png,0003,000
4,3,0003M1,Mega-Venusaur,Grass,Poison,6,3,6,2,5,...,NaN,NaN,venusaur-mega.png,NaN,0003,1,00031,poke_icon_0003_001_mf_n_00000000_f_n.png,0003,001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,888,0888,Zacian,Fairy,NaN,5,7,7,7,7,...,N,Yes,zacian.png,NaN,0888,0,08880,poke_icon_0888_000_uk_n_00000000_f_n.png,0888,000
1017,888,0888F1,Form Zacian,Fairy,Steel,5,9,9,8,8,...,N,Yes,zacian.png,NaN,0888,1,08881,poke_icon_0888_001_uk_n_00000000_f_n.png,0888,001
1018,889,0889,Zamazenta,Fighting,NaN,5,7,7,7,7,...,N,Yes,zamazenta.png,NaN,0889,0,08890,poke_icon_0889_000_uk_n_00000000_f_n.png,0889,000
1019,889,0889F1,Form Zamazenta,Fighting,Steel,5,7,7,7,7,...,N,Yes,zamazenta.png,NaN,0889,1,08891,poke_icon_0889_001_uk_n_00000000_f_n.png,0889,001


In [211]:
data = pd.read_csv(moves_path)
data['Damage1'] = data['Damage1'].str.capitalize()
data['Damage2'] = data['Damage2'].str.capitalize()
data['Accuracy1'] = data['Accuracy1'].str.capitalize()
data['Accuracy2'] = data['Accuracy2'].str.capitalize()
data['DmgType'] = data['DmgType'].str.capitalize()
data.to_csv('output.csv')


'Python {d["key"]}'